In [ ]:
import math
import numpy as np
import pandas as pd
from typing import Tuple, List, Optional, Generator

In [ ]:
def sign(n: int):
    return 1 if n > 0 else -1

def gcd_recursion(a: int, b: int) -> int:
    """ Find GCD by recursing on a mod b """
    if(b == 0):
        return sign(a) * a

    return gcd_recursion(b, a % b)

def gcd_euclidian(a: int, b: int) -> int:
    """ Euler algo """

    # a = q * b - r
    q = a // b
    r = a - q * b
    
    # Stopping condition: r = 0
    if r == 0:
        return b
    
    # next iteration b = r * q_2 + r
    return gcd_euclidian(b, r)

def gcd(a: int, b: int, fn = None) -> int:
    """  """
    # ensure a >= b
    a, b = (a, b) if a >= b else (b, a)

    fn = fn or gcd_euclidian
    return fn(a, b)

def extended_euclidian(a: int, b: int, state:Tuple[int, int, int] = None) -> int:
    """ Euler algo: find gcd(a, b) and x & y s.t. gcd(a, b) = x * a - y * b """

    # First time through, setup state, and ensure 'a' > 'b'
    state =  state or (((a, 1, 0), (b, 0, 1)) if a >= b else ((b, 1, 0), (a, 0, 1)))
   
    u = state[0]
    v = state[1]
    
    # Stopping condition: r = 0
    if v[0] <= 0:
        gcd = u[0]
        x = u[1] if a >= b else u[2]
        y = u[2] if a >= b else u[1]
        return (gcd, x, y)
    
    # a = q * b - r
    q = u[0] // v[0]
    w =  tuple(ui - q * vi for ui, vi in zip(u, v))
    state = (v, w)
    
    # next iteration b = r * q_2 + r
    return extended_euclidian(a, b, state)

def modulo_inverse(a, n):
    gcd, x, y = extended_euclidian(a, n)
    if gcd == 1:
        assert (a * x) % n == 1
        return x if x >= 0 else n + x 
    return np.NaN
    
def sieve_of_eratosthenes(n: int) -> Tuple[int,...]:
    """ Simple sieve to find primes up to n """
    n = n if n > 0 else -n
    if n < 2:
        return tuple()

    is_prime_sieve = np.full(n + 1, True)
    is_prime_sieve[0:2] = False
    is_prime_sieve[4::2] = False
    
    # Start with 2 and odd numbers from 3 to n
    sqrt_n = math.ceil(np.sqrt(n))
    for si in range(3, sqrt_n + 1, 2):
        if is_prime_sieve[si]:
            # Mark every multiple of si from si^2
            is_prime_sieve[si ** 2::si] = False
    return tuple(int(i) for i in np.flatnonzero(is_prime_sieve))

def primes_below(n: int) -> Generator[int, int, None]:
    """ """
    for p in sieve_of_eratosthenes(n - 1):
        yield p
    return

def next_prime(n: int):
    """ """
    for p in primes_below((math.trunc(np.sqrt(n)) + 1) ** 2):
        if p > n:
            return p
    return np.NaN

def is_prime(n: int) -> bool:
    """ check if is a prime by division: O(log(sqrt(n))) """
    for i in range(2, math.trunc(np.sqrt(n)) + 1):
        if n % i == 0:
            return  False
    return True

def ifactors(n: int) -> Generator[Tuple[int, int], int, None]:
    """   """
    n_abs = sign(n) * n
    half_n = n_abs // 2
    is_prime = True
    for p in primes_below(half_n + 1):
        if p > half_n:
            break
        
        
        if n_abs % p == 0:
            is_prime = False
            k = 1
            m = n_abs // p
            while m % p == 0:
                m = m // p
                k += 1
            yield (p, k)
    
    if is_prime:
        yield (n_abs, 1)
                
    return

def divisors(n: int) -> Tuple[int, ...]:
    """ """
    factors = tuple(ifactors(n))
    factor_primes = np.array([f[0] for f in factors] + [0,])
    # add one to every power, as we generate powers as (i mod factor_powers[j])
    factor_powers = np.array([f[1] for f in factors] + [0,]) + 1
    
    factors_count = len(factors)
    divisors_count = np.prod(factor_powers)

    # calc product of array of each prime factor to some power, varying from 0 to the max given from ifactors fn
    ds = sorted(int(np.prod([factor_primes[j] ** (i // np.prod(factor_powers[j - factors_count:]) % factor_powers[j])
                             for j in range(factors_count)]))
                for i in range(divisors_count))
    return tuple(ds)

def co_primes(n: int):
    """ """
    return set([a for a in range(1, n) if gcd(a, n) == 1])

def totient(n: int) -> int:
    """ Euler's phi function is the number of values less than a that are co-prime with n """
    return len(co_primes(n))

def order_of_powers(g: int, n: int) -> List[int]:
    """ g ^ k % n for k being co-prime with phi(n) """

    # order_of_powers = sorted(set([g ** k % n for k in co_primes(totient(n))]))
    # keep all calcs mod n to remove overflow errors
    ks = co_primes(totient(n))
    order_of_powers = set()
    g_k = 1
    for k in range(1, n):
        g_k = g_k * g  % n
        if k in ks:
            order_of_powers.add(g_k)
    return sorted(order_of_powers)

def order(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN

    a_k = 1
    for k in range(1, n):
        a_k = a_k * a  % n
        if a_k == 1:
            return k
    
    return np.NaN

def is_order_n(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN
    
    ord_n = totient(n)
    # we can do better than all k < n by only looking at divisors of totient(n)
    phi_n_divisors = divisors(ord_n)
    for k in phi_n_divisors:
        if a ** k % n == 1:
            return k == ord_n
    
    return np.NaN

def cyclic_group(a, n, op):
    group = set([(a ** k) % n for k in range(1, n)])
    return group

def primative_roots(n):
    # g is a primitive root modulo n if for every integer a coprime to n, there is some integer k for which gk ≡ a (mod n)
    
    # check n is form 2, 4, p^s, 2p^s, where s is any positive integer and p is an odd prime
    factors = [f for f in ifactors(n)]
    if any((len(factors) < 1 or 2 < len(factors),  
            (len(factors) == 2 and (factors[0][0] != 2 or factors[0][1] > 1)),
            (len(factors) == 1 and factors[0][0] == 2 and factors[0][1] > 2),
            (len(factors) == 1 and factors[0][0] < 2))):
        return [] # Exception("No primitive roots exist")
    
    # find smallest primative root
    ord_n = totient(n)
    g = None
    for a in co_primes(n):
        if order(a, n) == ord_n:
            g = a
            break
    
    # There are phi(phi(n)) roots: return all roots using factors co-prime with phi(n)
    prime_roots = order_of_powers(g, n)
     
    assert len(prime_roots) == totient(ord_n)
    assert all(g ** ord_n % n == 1 for g in prime_roots)
    return prime_roots

# Assignment 3

Stuart

a = 2353
b = 20837
c = 500551
d = 29213

Q6 = iii)

__Question 1 (10 marks):__

For the number $a$ given to you on page 1 answer the following:

* Show that a can be written as the sum of squares in two different ways. 
    
* Hence apply Euler’s method to factorise a.

In [ ]:
def sum_of_squares(n: int) -> Generator[Tuple[int, int], int, None]:
    """ Find two integers that when squared, add up to n """
    sqrt_n = int(math.sqrt(n))
    u = 1
    u2 = 1
    squares = {u2: u}
    for v in range(sqrt_n, sqrt_n // 2, -1):
        n_v2_diff = n - v ** 2
        while u2 < n_v2_diff:
            u += 1
            u2 = u ** 2
            squares[u2] = u
        if n_v2_diff in squares:
            yield (u, v)
    return

def diff_of_squares(n: int) -> Generator[Tuple[int, int], int, None]:
    """ Find two integers that when squared, the difference is n """
    sqrt_n = int(math.sqrt(n))
    squares = {}
    for v in range(1, sqrt_n // 2 + 1):
        squares[v ** 2] = v
    
    u = sqrt_n
    for i in range(1, n // 2):
        u = u + 1
        u2 = u ** 2
        squares[u2] = u
        u2_n_diff = u2 - n
        if u2_n_diff in squares:
            yield (u, squares[u2_n_diff])
    return

a = 2353
for u_v in sum_of_squares(a):
    print((u_v, u_v[0] ** 2 + u_v[1] ** 2))
for u_v in diff_of_squares(a):
    print((u_v, u_v[0] ** 2 - u_v[1] ** 2))# Write a function for Euler's method

__Question 2 (10 marks):__

Take the number $b$ assigned to you on page 1 and apply the gcd method to find its smallest factor.
    
What is the value of $P_0$ that you need to guarantee finding a factor given that $b$ is composite?

In [ ]:
b = 20837
print([f for f in ifactors(b)])

p0 = 1
for p in sieve_of_eratosthenes(int(np.sqrt(b))):
    p0 *= p
print(gcd(p0, b))


__Question 3 (10 marks):__

Take the number $c$ assigned to you on page and use the $p-1$ method to find one factor of $c$. You may assume that $c-1$ factorises into primes of size $< 100$.

In [ ]:
c = 500551

__Question 4 (10 marks)__

Take the number $d$ assigned to you on page and use the $p-1$ method to find both factors of $d$. You may NOT use the maple procedure provided in weblearn.

In [ ]:
d = 29213

__Question 5 (15 marks)__

Take the same number $d$ and now factorise using the Quadratic Sieve method. You may use Maple commands included in the week 10 workshop folder.

__Question 6 (15 marks)__

The Maple worksheet in Weblearn for this assignment shows part of an attempt to factorise $N = 9263 = 59*157$ using the Number Field Sieve. In this we claim the following:

&nbsp;&nbsp;&nbsp;&nbsp;$A = [0 , 1, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 2
    
&nbsp;&nbsp;&nbsp;&nbsp;$B = [-1 ,- 1, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 3
    
&nbsp;&nbsp;&nbsp;&nbsp;$C = [1, 0 , 1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 5  
    
&nbsp;&nbsp;&nbsp;&nbsp;$D = [1 , 1, -1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 11
    
&nbsp;&nbsp;&nbsp;&nbsp;$E = [1 , -2, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 17
    
&nbsp;&nbsp;&nbsp;&nbsp;$F = [3 , 0, -1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$  with norm = 23

We also derive a $48 x 15$ matrix $R$ consisting of values of $a$ and $b$ such that $a + 21b$ can be factorised using small primes and $[a, b, 0]$ can 
be factorised in Z(∛(-2) using just the primes ${A, B, C, D, E, F}$ and a unit element U = $[1, 1, 0]$.

&nbsp;&nbsp;&nbsp;&nbsp;Prove the statement *iii)* $C$ above allocated to you on page 1 using the definition of a norm.
    
&nbsp;&nbsp;&nbsp;&nbsp;Show that rows 23, 37, 41 and 45 of the matrix  form a linearly dependent set modulo 2
    
&nbsp;&nbsp;&nbsp;&nbsp;Hence find an equation of the form $u_2 – v_2$ such that $u_2 – v_2$ and $N$ have a common factor of $59$ thus 
factorising N. You may use the Maple procedure for multiplication in $\mathbb{Z}(\sqrt[3]{-2})$ to help you find $u$ and $v$.

__Q7__

Compare the methods for integer factorisation you have seen in the module and summarises their strengths and weaknesses, including the size of numbers that can be factorised, usability and whether or not they work for a broad range of numbers.  